In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0, "../src")

In [3]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

import fields
import variogram as vgm

In [4]:
# Analysis variables
timestamps = ["2016-07-01", "2019-08-01"]
tols = [100, 250]
timedeltas = [-1, -2]
space_lags = np.linspace(0, 4.5e3, 20)

timestamp = timestamps[1]
tol = tols[1]
timedelta = timedeltas[0]

# Initial guess for covariance params
cov_guess = [0.4, 1e3, 0.1]
cross_guess = [1e3, -0.1]

In [5]:
# Get data: 5-degree monthly average, cont. US
ds = xr.open_dataset("../data/exp_pro/OCO2_5deg_monthly_conus.nc")

ds_xco2 = ds[["xco2", "xco2_var"]]
ds_sif = ds[["sif", "sif_var"]]

In [6]:
mf = fields.MultiField(ds_xco2, ds_sif, timestamp, timedelta=timedelta, full_detrend=True, standardize_window=True)

/home/jj829/sif-data-fusion/conda-env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)
/home/jj829/sif-data-fusion/conda-env/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [7]:
vario_res, params_fit = vgm.variogram_analysis(
    mf,
    space_lags,
    tol,
    cov_guess,
    cross_guess
)

In [8]:
params_fit

{'xco2': array([6.13543523e-01, 1.53534617e+03, 1.35085004e-01]),
 'sif': array([8.52827464e-01, 9.39910532e+02, 2.77578253e-01]),
 'xco2:sif_back': array([ 4.e+03, -1.e+00]),
 'xco2:sif_forward': array([3.37597056e+03, 2.08103661e-01])}

In [11]:
vario_res

{'xco2':             lag           xco2  counts   wls_fit
 0      0.000000   0.000000e+00   864.0  0.157077
 1    473.684211   2.257337e-01  4296.0  0.208257
 2    710.526316  1.067802e+297  5472.0  0.255170
 3    947.368421   2.866709e-01  7584.0  0.302834
 4   1184.210526  1.134297e+297  8448.0  0.345725
 5   1421.052632  1.202345e+297  9120.0  0.381491
 6   1657.894737  1.232549e+297  9576.0  0.409790
 7   1894.736842   3.992670e-01  9648.0  0.431332
 8   2131.578947   4.464735e-01  9288.0  0.447254
 9   2368.421053  1.397634e+297  9504.0  0.458749
 10  2605.263158  1.480723e+297  9168.0  0.466892
 11  2842.105263  1.505572e+297  8952.0  0.472571
 12  3078.947368   5.146461e-01  6648.0  0.476478
 13  3315.789474   5.083658e-01  7368.0  0.479137
 14  3552.631579   4.846966e-01  6672.0  0.480928
 15  3789.473684  1.769942e+297  5304.0  0.482125
 16  4026.315789   4.961000e-01  4344.0  0.482918,
 'sif':             lag            sif   counts   wls_fit
 0      0.000000   0.000000e+00  

In [21]:
xco2_params, df_vgm["xco2_fit"] = vgm.fit_variogram_wls(df_vgm["lag"], df_vgm["xco2"], df_vgm["xco2_counts"], vgm_guess)
sif_params, df_vgm["sif_fit"] = vgm.fit_variogram_wls(df_vgm["lag"], df_vgm["sif"], df_vgm["sif_counts"], vgm_guess)

sigmas = [xco2_params[0], sif_params[0]]
cross_params_back, df_cov["cross_fit_back"] = vgm.fit_variogram_wls(df_cov["lag"], df_cov["xco2:sif_back"], df_cov["xco2:sif_counts_back"], cross_guess, sigmas=sigmas)
cross_params_forward, df_cov["cross_fit_forward"] = vgm.fit_variogram_wls(df_cov["lag"], df_cov["xco2:sif_forward"], df_cov["xco2:sif_counts_forward"], cross_guess, sigmas=sigmas)

NameError: name 'df_vgm' is not defined